In [2]:
# Install dependencies
!pip install langchain faiss-cpu sentence-transformers gpt4all

import json
import faiss
import pickle
from sentence_transformers import SentenceTransformer
from gpt4all import GPT4All
from IPython.display import display
import ipywidgets as widgets

# ---- Step 1: Load JSON Knowledge Base ----
with open("/content/train.json") as f:
    train_data = json.load(f)

with open("/content/test (1).json") as f:
    test_data = json.load(f)

data = train_data + test_data

chunks = []
for item in data:
    q = item.get("question", "")
    a = " ".join(item.get("answers", []))
    context = item.get("context", "")
    text = f"Q: {q}\nContext: {context}\nA: {a}"
    chunks.append(text)

print(f" Loaded {len(chunks)} knowledge chunks")

# ---- Step 2: Embeddings + FAISS ----
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(chunks, convert_to_numpy=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

pickle.dump({"chunks": chunks}, open("qa_store.pkl", "wb"))
faiss.write_index(index, "qa_index.idx")

# ---- Step 3: Reload FAISS ----
index = faiss.read_index("qa_index.idx")
store = pickle.load(open("qa_store.pkl", "rb"))
chunks = store["chunks"]

def retrieve(query, k=3):
    q_emb = embed_model.encode([query])
    D, I = index.search(q_emb, k)
    return [chunks[i] for i in I[0]], D[0][0]

# ---- Step 4: Load Local LLM ----
llm = GPT4All("gpt4all-falcon-newbpe-q4_0.gguf")  # first run downloads model

def ask(query, threshold=0.45):
    results, best_distance = retrieve(query, k=3)
    similarity = 1 - (best_distance / 2)
    context = "\n".join(results)

    if similarity > threshold:
        return " Report-based Answer:\n" + context
    else:
        prompt = f"""The knowledge base does not contain this info.
Provide a helpful physiotherapy suggestion.

Question: {query}
Answer:"""
        response = llm.generate(prompt, max_tokens=200)
        return " General Physiotherapy Advice:\n" + response.strip()

# ---- Step 5: Interactive Chat UI ----
chat_history = widgets.Output()
input_box = widgets.Text(placeholder="Ask your physiotherapy question...")
send_button = widgets.Button(description="Send", button_style='success')

def on_send(b):
    with chat_history:
        query = input_box.value
        if query.strip():
            print(f"\n You: {query}")
            answer = ask(query)
            print(f" Bot: {answer}")
        input_box.value = ""  # clear box

send_button.on_click(on_send)
display(input_box, send_button, chat_history)


 Loaded 2876 knowledge chunks


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading: 100%|██████████| 4.21G/4.21G [01:04<00:00, 65.1MiB/s]
Verifying: 100%|██████████| 4.21G/4.21G [00:15<00:00, 275MiB/s]


Text(value='', placeholder='Ask your physiotherapy question...')

Button(button_style='success', description='Send', style=ButtonStyle())

Output()